# Importing Libraries

## Goal 1: Connect Gmail in read only mode and fetch unread or recent emails

In [156]:
import imaplib
import pprint
import email 
from email.policy import default

In [157]:
import yaml

In [158]:
with open("CONFIDENTIAL.yml") as f:
    content = f.read()

In [159]:
credentials = yaml.load(content, Loader=yaml.FullLoader)

In [160]:
user, password = credentials["user"], credentials["password"]

In [161]:
imap_host = 'imap.gmail.com'
imap_user = user
imap_pass = password

In [162]:
imap = imaplib.IMAP4_SSL(imap_host)

In [163]:
imap.login(imap_user, imap_pass)

('OK', [b'bilalrukundi1658@gmail.com authenticated (Success)'])

Gmail requires you to either create a 

"app password" 
or to enable
"imap"

in account settings in order to login into your account so no third party apps can login 
with just the username/email and password.

In [164]:
imap.select('inbox') 

('OK', [b'310'])

In [165]:
msgs = []

In [166]:
tmp, data = imap.search(None, 'ALL')

In [167]:
mail_id = data[0].split()

In [168]:
for num in mail_id:
    status, msg_data = imap.fetch(num,"(RFC822)")